# A Cobblemon Pasture Loot Generator

This script is meant to create loot tables in `.json` for Minecraft from `.csv` raw data.

We first install the libraries using `pip`

In [28]:
%pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


We then import the libraries in order to use them in the code.

In [29]:
import csv
import json
import os
import re
from difflib import get_close_matches

We define the relative paths of the folders. The csv files needs to be in the same folder as the script for convinience

In [30]:
# Define file paths
drops_csv_path = 'drops.csv'
rolls_csv_path = 'rolls.csv'
output_dir = '../data/pasturecollector/loot_tables/gameplay/pasture_collector/species/'

We read the rarity chances from the rolls file

In [31]:

# Read rolls.csv to get the chances
rarity_chances = {}
with open(rolls_csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        rarity_chances[row['rarity']] = float(row['chance'].strip('%')) / 100

We read the csv for items and load them as dictionaries.

In [32]:
# Define file paths for items CSVs
cobblemon_items_csv_path = 'cobblemon_items.csv'
minecraft_items_csv_path = 'minecraft_items.csv'

# Load items from CSVs into a common dictionary
items_dict = {}

# Load Cobblemon items
with open(cobblemon_items_csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        items_dict[row['Item name'].lower()] = row['ID']

# Load Minecraft items
with open(minecraft_items_csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        items_dict[row['Item name'].lower()] = row['ID']

print("Items dictionary loaded successfully.")
print(items_dict)

Items dictionary loaded successfully.
{'absorb bulb': 'cobblemon:absorb_bulb', 'air balloon': 'cobblemon:air_balloon', 'aspear berry': 'cobblemon:aspear_berry', 'auspicious armor': 'cobblemon:auspicious_armor', 'babiri berry': 'cobblemon:babiri_berry', 'berry juice': 'cobblemon:berry_juice', 'big root': 'cobblemon:big_root', 'black augurite': 'cobblemon:black_augurite', 'black belt': 'cobblemon:black_belt', 'black glasses': 'cobblemon:black_glasses', 'black sludge': 'cobblemon:black_sudge', 'bluk berry': 'cobblemon:bluk_berry', 'bright powder': 'cobblemon:bright_powder', 'cell battery': 'cobblemon:cell_battery', 'charcoal stick': 'cobblemon:charcoal_stick', 'charti berry': 'cobblemon:charti_berry', 'chesto berry': 'cobblemon:chesto_berry', 'chilan berry': 'cobblemon:chilan_berry', 'chople berry': 'cobblemon:chople_berry', 'cleanse tag': 'cobblemon:cleanse_tag', 'coba berry': 'cobblemon:coba_berry', 'colbur berry': 'cobblemon:colbur_berry', 'dawn stone': 'cobblemon:dawn_stone', 'deep se

That's the big one ! We read the drops and for each line we make a json file. For each pokemon the rarity will dictate the random tick chance. For each drop we try to check if it's a valid id, if not we try to match the name perfectly to one in the item list. If all fails we try a near match and give a warning. If it's too far off we throw an error.

In [33]:
from difflib import get_close_matches


# Read drops.csv and generate JSON files
with open(drops_csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        pokemon = row['Pokemon'].lower()
        rarity = row['Rarity']
        chance = rarity_chances.get(rarity, 0.0)
        print(f"Processing {pokemon} with rarity {rarity} and chance {chance} ...")
        entries = []
        for i in range(1, 6):
            drop = row[f'Drops{i}']
            print(f"Drop {i}: {drop}")
            if drop:
                match = re.match(r'(.+?)\s+([\d\-\.%]+)$', drop)
                if match:
                    item = match.group(1).lower()
                    count = match.group(2)
                else:
                    item = drop.lower()
                    count = '1'
                
                if item in items_dict.values():
                    item_id = item
                else:
                    item_id = items_dict.get(item)
                if not item_id:
                    close_matches = get_close_matches(item, items_dict.keys(), n=1, cutoff=0.6)
                    if close_matches:
                        item_id = items_dict[close_matches[0]]
                        print(f"Warning: Item '{item}' not found. Using closest match '{close_matches[0]}' instead.")
                    else:
                        print(f"Error: Item '{item}' not found and no close match available.")
                        continue
                if '-' in count:
                    min_count, max_count = map(int, count.split('-'))
                    count = {"min": min_count, "max": max_count}
                elif '%' in count:
                    count = float(count.strip('%')) / 100
                else:
                    count = int(count)
                entries.append({
                    "type": "minecraft:item",
                    "name": item,
                    "functions": [{"function": "minecraft:set_count", "count": count}]
                })
        
        loot_table = {
            "pools": [{
                "rolls": 1,
                "entries": entries,
                "conditions": [{"condition": "minecraft:random_chance", "chance": chance}]
            }]
        }
        
        # Write JSON file
        output_path = os.path.join(output_dir, f'{pokemon}.json')
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Write JSON file
        with open(output_path, mode='w') as json_file:
            json.dump(loot_table, json_file, indent=2)

print("JSON files generated successfully.")

Processing bulbasaur with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:melon_seeds 0-1
Drop 2: cobblemon:miracle_seed 5%
Drop 3: 
Drop 4: 
Drop 5: 
Processing ivysaur with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:melon_seeds 0-2
Drop 2: cobblemon:miracle_seed 10%
Drop 3: 
Drop 4: 
Drop 5: 
Processing venusaur with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:wheat_seeds 1
Drop 2: cobblemon:coba_berry 2.5%
Drop 3: 
Drop 4: 
Drop 5: 
Processing charmander with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:blaze_powder 0-1
Drop 2: cobblemon:charcoal_stick 5%
Drop 3: 
Drop 4: 
Drop 5: 
Processing charmeleon with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:blaze_powder 0-2
Drop 2: cobblemon:charcoal_stick 10%
Drop 3: 
Drop 4: 
Drop 5: 
Processing charizard with rarity ultra-rare and chance 0.03 ...
Drop 1: minecraft:blaze_powder 0-3
Drop 2: cobblemon:charcoal_stick 25%
Drop 3: 
Drop 4: 
Drop 5: 
Processing squirtle with rarity ultra